In [ ]:
#| default_exp 02_ngame-for-wikiseealsotitles-20250123

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import os,torch,json, torch.multiprocessing as mp, joblib, numpy as np, scipy.sparse as sp

from xcai.basics import *
from xcai.models.PPP0XX import DBT009,DBT011

In [ ]:
os.environ['WANDB_MODE'] = 'disabled'

In [ ]:
#| export
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
os.environ['WANDB_PROJECT'] = 'mogicX_01-wikiseealsotitles'

## Setup

In [ ]:
output_dir = '/scratch/scai/phd/aiz218323/outputs/mogicX/02_ngame-for-wikiseealsotitles-20250123'

config_file = '/home/scai/phd/aiz218323/scratch/datasets/benchmarks/20250123-LF-WikiSeeAlsoTitles-320K/configs/data.json'
config_key = 'data'

mname = 'sentence-transformers/msmarco-distilbert-dot-v5'

In [ ]:
output_dir = '/scratch/scai/phd/aiz218323/outputs/mogicX/02_ngame-for-wikiseealsotitles-20250123'

config_file = '/home/scai/phd/aiz218323/scratch/datasets/wikipedia/20250123/LF-WikiSeeAlsoTitles-320K/configs/data_category_old-combined-new.json'
config_key = 'data_category_old-combined-new'

mname = 'sentence-transformers/msmarco-distilbert-dot-v5'

In [ ]:
do_train_inference = True
do_test_inference = True

save_train_inference = False
save_test_inference = False

save_representation = False

use_sxc_sampler, only_test = True, False

pkl_dir = '/scratch/scai/phd/aiz218323/datasets/processed/'

In [ ]:
pkl_file = f'{pkl_dir}/mogicX/wikiseealsotitles-20250123_data_distilbert-base-uncased'
pkl_file = f'{pkl_file}_sxc' if use_sxc_sampler else f'{pkl_file}_xcs'
if only_test: pkl_file = f'{pkl_file}_only-test'
pkl_file = f'{pkl_file}.joblib'

In [ ]:
pkl_file = f'{pkl_dir}/mogicX/wikiseealsotitles-20250123_data-category-old-combined-new_distilbert-base-uncased'
pkl_file = f'{pkl_file}_sxc' if use_sxc_sampler else f'{pkl_file}_xcs'
if only_test: pkl_file = f'{pkl_file}_only-test'
pkl_file = f'{pkl_file}.joblib'

In [ ]:
do_inference = do_train_inference or do_test_inference or save_train_inference or save_test_inference or save_representation

In [ ]:
pkl_file

'/scratch/scai/phd/aiz218323/datasets/processed//mogicX/wikiseealsotitles-20250123_data-category-old-combined-new_distilbert-base-uncased_sxc.joblib'

In [ ]:
%%time
os.makedirs(os.path.dirname(pkl_file), exist_ok=True)
block = build_block(pkl_file, config_file, use_sxc_sampler, config_key, do_build=True)

CPU times: user 8min 31s, sys: 1min 27s, total: 9min 59s
Wall time: 6min 9s


In [ ]:
args = XCLearningArguments(
    output_dir=output_dir,
    logging_first_step=True,
    per_device_train_batch_size=800,
    per_device_eval_batch_size=800,
    representation_num_beams=200,
    representation_accumulation_steps=10,
    save_strategy="steps",
    eval_strategy="steps",
    eval_steps=10,
    save_steps=10,
    save_total_limit=5,
    num_train_epochs=300,
    predict_with_representation=True,
    representation_search_type='BRUTEFORCE',
    adam_epsilon=1e-6,                                                                                                                                          warmup_steps=100,
    weight_decay=0.01,
    learning_rate=2e-5,

    group_by_cluster=True,
    num_clustering_warmup_epochs=10,
    num_cluster_update_epochs=5,
    num_cluster_size_update_epochs=25,
    clustering_type='EXPO',
    minimum_cluster_size=2,
    maximum_cluster_size=1600,

    metric_for_best_model='P@1',
    load_best_model_at_end=True,
    target_indices_key='plbl2data_idx',
    target_pointer_key='plbl2data_data2ptr',

    use_encoder_parallel=True,
    max_grad_norm=None,
    fp16=True,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
comet_ml version 3.39.1 is installed, but version 3.43.2 or higher is required. Please update comet_ml to the latest version to enable Comet logging with pip install 'comet-ml>=3.43.2'.


In [ ]:
metric = PrecReclMrr(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                     pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200], mk=[5, 10, 20])

In [ ]:
def model_fn(mname, bsz):
    model = DBT009.from_pretrained(mname, bsz=bsz, tn_targ=5000, margin=0.3, tau=0.1, n_negatives=10, 
                                   apply_softmax=True, use_encoder_parallel=True)
    return model

def init_fn(model): 
    model.init_dr_head()

In [ ]:
bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()

model = load_model(args.output_dir, model_fn, {"mname": mname, "bsz": bsz}, init_fn, do_inference=do_inference, use_pretrained=True)

Some weights of DBT009 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-dot-v5 and are newly initialized: ['encoder.dr_layer_norm.bias', 'encoder.dr_layer_norm.weight', 'encoder.dr_projector.bias', 'encoder.dr_projector.weight', 'encoder.dr_transform.bias', 'encoder.dr_transform.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
learn = XCLearner(
    model=model,
    args=args,
    train_dataset=block.train.dset,
    eval_dataset=block.test.dset,
    data_collator=block.collator,
    compute_metrics=metric,
)

main(learn, input_args, n_lbl=block.n_lbl)

## Driver

In [ ]:
#| export
if __name__ == '__main__':
    output_dir = '/scratch/scai/phd/aiz218323/outputs/mogicX/02_ngame-for-wikiseealsotitles-20250123'

    config_file = '/home/scai/phd/aiz218323/scratch/datasets/benchmarks/20250123-LF-WikiSeeAlsoTitles-320K/configs/data.json'
    config_key = 'data'
    
    mname = 'sentence-transformers/msmarco-distilbert-dot-v5'

    input_args = parse_args()

    pkl_file = f'{input_args.pickle_dir}/mogicX/wikiseealsotitles-20250123_data_distilbert-base-uncased'
    pkl_file = f'{pkl_file}_sxc' if input_args.use_sxc_sampler else f'{pkl_file}_xcs'
    if input_args.only_test: pkl_file = f'{pkl_file}_only-test'
    pkl_file = f'{pkl_file}.joblib'

    do_inference = input_args.do_train_inference or input_args.do_test_inference or input_args.save_train_prediction or input_args.save_test_prediction or input_args.save_representation

    os.makedirs(os.path.dirname(pkl_file), exist_ok=True)
    block = build_block(pkl_file, config_file, input_args.use_sxc_sampler, config_key, do_build=input_args.build_block, only_test=input_args.only_test)

    args = XCLearningArguments(
        output_dir=output_dir,
        logging_first_step=True,
        per_device_train_batch_size=800,
        per_device_eval_batch_size=800,
        representation_num_beams=200,
        representation_accumulation_steps=10,
        save_strategy="steps",
        evaluation_strategy="steps",
        eval_steps=5000,
        save_steps=5000,
        save_total_limit=5,
        num_train_epochs=300,
        predict_with_representation=True,
        representation_search_type='BRUTEFORCE',
        adam_epsilon=1e-6,                                                                                                                                          warmup_steps=100,
        weight_decay=0.01,
        learning_rate=2e-4,
    
        group_by_cluster=True,
        num_clustering_warmup_epochs=10,
        num_cluster_update_epochs=5,
        num_cluster_size_update_epochs=25,
        clustering_type='EXPO',
        minimum_cluster_size=2,
        maximum_cluster_size=1600,
    
        metric_for_best_model='P@1',
        load_best_model_at_end=True,
        target_indices_key='plbl2data_idx',
        target_pointer_key='plbl2data_data2ptr',
    
        use_encoder_parallel=True,
        max_grad_norm=None,
        fp16=True,
    )

    def model_fn(mname, bsz):
        model = DBT009.from_pretrained(mname, bsz=bsz, tn_targ=5000, margin=0.3, tau=0.1, n_negatives=10, 
                                       apply_softmax=True, use_encoder_parallel=True)
        return model
    
    def init_fn(model): 
        model.init_dr_head()

    metric = PrecReclMrr(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                         pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200], mk=[5, 10, 20])
    
    bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()

    model = load_model(args.output_dir, model_fn, {"mname": mname, "bsz": bsz}, init_fn, do_inference=do_inference, use_pretrained=input_args.use_pretrained)
    
    learn = XCLearner(
        model=model,
        args=args,
        train_dataset=block.train.dset,
        eval_dataset=block.test.dset,
        data_collator=block.collator,
        compute_metrics=metric,
    )
    
    main(learn, input_args, n_lbl=block.n_lbl)
    

## Analysis

In [ ]:
def get_pickle_filename(pkl_dir, pkl_file, use_sxc_sampler, only_test):
    pkl_file = f'{pkl_dir}/{pkl_file}'
    pkl_file = f'{pkl_file}_sxc' if use_sxc_sampler else f'{pkl_file}_xcs'
    if only_test: pkl_file = f'{pkl_file}_only-test'
    pkl_file = f'{pkl_file}.joblib'
    return pkl_file
    

In [ ]:
from xcai.analysis import *

In [ ]:
use_sxc_sampler, only_test = True, False

In [ ]:
pkl_dir = '/scratch/scai/phd/aiz218323/datasets/processed/mogicX/'

pkl_file_1 = get_pickle_filename(pkl_dir, 'wikiseealsotitles_data_distilbert-base-uncased', use_sxc_sampler, only_test)
pkl_file_2 = get_pickle_filename(pkl_dir, 'wikiseealsotitles-20250123_data-category-old-combined-new_distilbert-base-uncased', use_sxc_sampler, only_test)

In [ ]:
pred_dir = '/home/scai/phd/aiz218323/scratch/outputs/mogicX/02_ngame-for-wikiseealsotitles-20250123-001/predictions'

pred_1, block_1 = load_prediction_and_block(pkl_file_1, 'wikiseealsotitles', 'data', f'{pred_dir}/test_predictions_wikiseealsotitles_data_distilbert-base-uncased.npz', 
                                            use_sxc_sampler)

pred_2, block_2 = load_prediction_and_block(pkl_file_2, '/home/scai/phd/aiz218323/scratch/datasets/wikipedia/20250123/LF-WikiSeeAlsoTitles-320K/configs/data_category_old-combined-new.json', 
                                            'data_category_old-combined-new', f'{pred_dir}/test_predictions_wikiseealsotitles-20250123_data-category-old-combined-new_distilbert-base-uncased.npz',
                                            use_sxc_sampler)

pred_block_1 = TextDataset(PredictionBlock(block_1.test.dset, pred_1))
pred_block_2 = TextDataset(PredictionBlock(block_2.test.dset, pred_2))

In [ ]:
idxs = Indices.metric(pred_block_1.dset, pred_block_2.dset, topk=1000)

In [ ]:
cmp = CompareDataset(pred_block_1, pred_block_2)
cmp.show(idxs=idxs[:10], k=25)

1. data_input_text : Frisian Water Line
2. data_input_text : Frisian Waterline
1. lbl2data_input_text : ['Old / New Dutch Waterline', 'Grebbe line', 'IJssel Line', 'Maas Line', 'Peel-Raam Line', 'Defence lines of the Netherlands']
2. lbl2data_input_text : ['Peel-Raam Line', 'Grebbe Line', 'Defence lines of the Netherlands', 'Maas Line', 'Grebbe line', 'IJssel Line', 'Dutch Water Line', 'Old / New Dutch Waterline']
1. pred2data_input_text : ['Defence lines of the Netherlands', 'Peel-Raam Line', 'Old / New Dutch Waterline', 'IJssel Line', 'Grebbe line', 'North Frisian Barrier Island', 'North Frisia', 'Maas Line', 'Defence Line of Amsterdam', 'East Frisia']
2. pred2data_input_text : ['North Frisia', 'North Frisian Barrier Island', 'East Frisia', 'Frisian Islands', 'East Frisian Islands', 'Frisia', 'North Frisia (disambiguation)', 'North Frisians', 'East Frisian Low Saxon', 'West Frisians']

1. data_input_text : Franco-Belgian comics
2. data_input_text : Bande dessinée
1. lbl2data_input_te

In [ ]:
import os

save_dir = '/home/scai/phd/aiz218323/scratch/outputs/mogicX/02_ngame-for-wikiseealsotitles-20250123-001/examples'
os.makedirs(save_dir, exist_ok=True)

In [ ]:
fname = f'{save_dir}/wikiseealsotitles-V-wikiseealsotitles-20250123_data-category-old-combined-new_distilbert-base-uncased.txt'

In [ ]:
cmp.dump_txt(fname, idxs=idxs)